<a href="https://colab.research.google.com/github/chetools/CHE4061_Fall2024/blob/main/MESHDistillation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget -N -q https://raw.githubusercontent.com/chetools/chetools/main/tools/che5.ipynb -O che5.ipynb
!pip install importnb

In [ ]:
from importnb import Notebook
with Notebook():
    from che5 import Props

import numpy as np
import jax
import jax.numpy as jnp
jax.config.update("jax_enable_x64", True)
from scipy.optimize import root_scalar
from scipy.optimize import root
from scipy.special import expit, logit
from plotly.subplots import make_subplots
from scipy.interpolate import Akima1DInterpolator
np.set_printoptions(precision=1)

In [ ]:
R=8.314
p=Props(['Ethanol', 'Water'])

In [ ]:
def gamma(x,T):
    tau = p.NRTL_A + p.NRTL_B/T + p.NRTL_C*np.log(T) + p.NRTL_D*T
    G=np.exp(-p.NRTL_alpha*tau)
    xG = x@G
    xtauG_xG = (x@(tau*G))/xG
    return np.exp(xtauG_xG + x@((G*(tau - xtauG_xG[None,:])/xG[None,:]).T))

In [ ]:
def dewP_ideal(y, T):
    P=1./(np.sum(y/p.Pvap(T)))
    return P, y*P/p.Pvap(T)

def dewT_ideal(y, P):

    def P_dev(T):
        return dewP_ideal(y, T)[0] - P
    T = root(P_dev, 300.).x[0]

    return T,  y*P/p.Pvap(T)

In [ ]:
def bubbleP_NRTL(x, T):
    Pi= x*gamma(x,T)*p.Pvap(T)
    P=np.sum(Pi)
    return P, Pi/P

def bubbleT_NRTL(x, P):

    def f(T):
        return bubbleP_NRTL(x,T)[0]-P

    #mole-fraction weighted boiling points of each component at P
    #boiling points determined via Clausius Clapeyron, using the Hvap at the normal bp
    #for each component.  p.Hvap returns the heat of vaporization of all components for each
    #temperature if an array of temperatures is given.
    Tguess=np.dot(x,1/(1/p.Tbn-np.log(P/101325)*R/np.diagonal(p.Hvap(p.Tbn))))

    res=root_scalar(f, x0=Tguess, method='secant')
    if not(res.converged):
        return "FAIL", res
    T=root_scalar(f, x0=Tguess, method='secant').root
    Pi= x*gamma(x,T)*p.Pvap(T)
    P=np.sum(Pi)

    return T, Pi/P



In [ ]:
P=101325
x1s=np.linspace(0,1,101)
Ts=[]
y1s=[]
for x1 in x1s:
    T, (y1,y2) = bubbleT_NRTL(np.array([x1, 1-x1]), P)
    Ts.append(T)
    y1s.append(y1)




In [ ]:
def dewP_NRTL(y, T):
    def f(vec):
        P = vec[0]
        x = expit(vec[1:])  #Ensures that mole fractions are between 0 and 1

        fug_eqs = x*gamma(x,T) * p.Pvap(T)  - y*P
        xsum_eq = 1. - np.sum(x)

        return np.r_[fug_eqs, xsum_eq]

    #Assume ideal liquid dewP calculation for initial guess of P and liquid phase composition
    Pguess, xguess= dewP_ideal(z,T)

    #f (function to zero) maps values from -inf to inf, to values between 0 and 1
    #so xguess is mapped via logit which is the inverse function of expit
    v0 = np.r_[Pguess, logit(xguess)]
    res=root(f, v0)
    if not(res.success):
        return "FAILURE", res
    return res.x[0], expit(res.x[1:])

In [ ]:
def dewT_NRTL(y, P):
    def f(vec):
        T = vec[0]
        x = expit(vec[1:])  #Ensures that mole fractions are between 0 and 1

        fug_eqs = x*gamma(x,T) * p.Pvap(T)  - y*P
        xsum_eq = 1. - np.sum(x)

        return np.r_[fug_eqs, xsum_eq]

    #Assume ideal liquid dewP calculation for initial guess of P and liquid phase composition
    Tguess, xguess= dewT_ideal(z,P)

    #f (function to zero) maps values from -inf to inf, to values between 0 and 1
    #so xguess is mapped via logit which is the inverse function of expit
    v0 = np.r_[Tguess, logit(xguess)]
    res=root(f, v0)
    if not(res.success):
        return "FAILURE", res
    return res.x[0], expit(res.x[1:])

In [ ]:
def flash_idealPT(z, P, T):

    K=p.Pvap(T)/P
    def rachford(VF):
        return np.sum(z*(K-1)/(VF*(K-1) +1))

    res=root_scalar(rachford, bracket=(0,1))
    VF = res.root
    x=z/(1-VF + K *VF)
    y=K*x
    return x, y, VF

In [ ]:
def flash_NRTL_PT(z, P, T, maxiter = 100, tol=1e-12):

    dewP, dewx = dewP_NRTL(z, T)
    bubbleP, bubbley = bubbleP_NRTL(z,T)

    xguess = (P-dewP)/(bubbleP-dewP) * (z - dewx) +  dewx

    for i in range(maxiter):
        K=gamma(xguess,T)*p.Pvap(T)/P

        def rachford(VF):
            return np.sum(z*(K-1)/(VF*(K-1) +1))

        res=root_scalar(rachford, bracket=(0,1))
        VF = res.root
        x=z/(1-VF + K *VF)
        if (np.linalg.norm(xguess-x)<tol):
            break
        xguess = x

    y=K*x
    return x, y, VF, i

In [ ]:
def flash_NRTL_PT(z, P, T, maxiter = 100, tol=1e-12):

    dewP, dewx = dewP_NRTL(z, T)
    bubbleP, bubbley = bubbleP_NRTL(z,T)

    xguess = (P-dewP)/(bubbleP-dewP) * (z - dewx) +  dewx

    for i in range(maxiter):
        K=gamma(xguess,T)*p.Pvap(T)/P

        def rachford(VF):
            return np.sum(z*(K-1)/(VF*(K-1) +1))

        res=root_scalar(rachford, bracket=(0,1))
        VF = res.root
        x=z/(1-VF + K *VF)
        if (np.linalg.norm(xguess-x)<tol):
            break
        xguess = x

    y=K*x
    return x, y, VF, i



In [ ]:
Nc = p.Mw.size
Ftot = 2.
z = np.array([0.5,0.5])
P = 1e5
dewT, _ = dewT_NRTL(z, P)
bubbleT, _ = bubbleT_NRTL(z,P)
feedT = (bubbleT + dewT)/2
x,y, vf, _ = flash_NRTL_PT(z, P, feedT)
feedH=p.Hv(Ftot*vf*y, feedT) + p.Hl(Ftot*(1-vf)*x,feedT)

In [ ]:
D = 1.
B = Ftot - D
R = 5.
Ns = 8
Nf = 4

unk = np.zeros((Ns, 2*Nc+1))
Ltot_rec = R*D
Ltot_strip = Ltot_rec + Ftot*(1-vf)
Vtot_rec = R*D + D
Vtot_strip = Vtot_rec - Ftot*vf
unk[:Nf,:Nc] = Ltot_rec*y
unk[Nf:-1,:Nc] = Ltot_strip*x
unk[-1,:Nc] = B*x
unk[:Nf,Nc:2*Nc ] = Vtot_rec*y
unk[Nf:,Nc:2*Nc ] = Vtot_strip*x
unk[:,-1]=np.linspace(bubbleT_NRTL(y,P)[0],dewT_NRTL(x,P)[0],Ns)

In [ ]:
unk

array([[2.8e+00, 2.2e+00, 3.4e+00, 2.6e+00, 3.5e+02],
       [2.8e+00, 2.2e+00, 3.4e+00, 2.6e+00, 3.5e+02],
       [2.8e+00, 2.2e+00, 3.4e+00, 2.6e+00, 3.6e+02],
       [2.8e+00, 2.2e+00, 3.4e+00, 2.6e+00, 3.6e+02],
       [1.4e+00, 4.0e+00, 1.1e+00, 3.3e+00, 3.6e+02],
       [1.4e+00, 4.0e+00, 1.1e+00, 3.3e+00, 3.6e+02],
       [1.4e+00, 4.0e+00, 1.1e+00, 3.3e+00, 3.6e+02],
       [2.6e-01, 7.4e-01, 1.1e+00, 3.3e+00, 3.7e+02]])

In [ ]:
def stage1(vec, vec2, refluxT):
    T,T2 = vec[-1], vec2[-1]
    L,V = np.split(vec[:-1],2)
    L2,V2 = np.split(vec2[:-1],2)
    x = L/np.sum(L)
    y = V/np.sum(V)

    MB = (V2 + R*D*y - V - L)/Ftot
    EQ = x*p.NRTL_gamma(x,T)*p.Pvap(T)/P - y
    EB = (p.Hv(V2, T2) + p.Hl(R*D*y, refluxT) - p.Hv(V, T) - p.Hl(L, T))/feedH

    return jnp.r_[MB, EQ, EB]

def stage(vec1, vec, vec2, f, fH):
    T1, T,T2 = vec1[-1], vec[-1], vec2[-1]
    L1,V1 = np.split(vec1[:-1],2)
    L,V = np.split(vec[:-1],2)
    L2,V2 = np.split(vec2[:-1],2)
    x = L/np.sum(L)
    y = V/np.sum(V)

    MB = (f + V2 + L1 - V - L)/Ftot
    EQ = x*p.NRTL_gamma(x,T)*p.Pvap(T)/P - y
    EB = (fH + p.Hv(V2, T2) + p.Hl(L1, T1) - p.Hv(V, T) - p.Hl(L, T))/feedH

    return jnp.r_[MB, EQ, EB]


def stageN(vec1, vec):
    T1, T = vec1[-1], vec[-1]
    L1,V1 = np.split(vec1[:-1],2)
    L,V = np.split(vec[:-1],2)
    x = L/np.sum(L)
    y = V/np.sum(V)

    MB = (L1 - V - L)/Ftot
    EQ = x*p.NRTL_gamma(x,T)*p.Pvap(T)/P - y
    MB2 = (np.sum(L1) - np.sum(V) - B)/Ftot

    return jnp.r_[MB, EQ, MB2]

In [ ]:
es = np.zeros((Ns, 2*Nc+1))


In [ ]:
L,V = np.split(unk[0,:-1],2)
y=V/np.sum(V)
es[0] = stage1(unk[0], unk[1], bubbleT_NRTL(y, P)[0])
for i in range(1, Ns-1):
    if i==Nf:
        es[i]= stage(unk[i-1], unk[i], unk[i+1], Ftot*z, feedH)
    else:
        es[i]= stage(unk[i-1], unk[i], unk[i+1], 0., 0.)


In [ ]:
es

array([[ 4.2e-15,  3.3e-15,  1.2e-01, -1.2e-01, -1.1e-03],
       [-2.2e-16,  2.2e-16,  1.7e-01, -9.3e-02,  1.1e-03],
       [-2.2e-16,  2.2e-16,  2.2e-01, -6.7e-02,  1.1e-03],
       [-1.1e+00,  3.3e-01,  2.8e-01, -3.9e-02, -7.4e-01],
       [ 1.2e+00, -4.2e-01,  4.2e-01, -2.1e-01,  7.3e-01],
       [ 1.1e-16,  4.4e-16,  4.7e-01, -1.8e-01,  1.3e-03],
       [ 1.1e-16,  4.4e-16,  5.2e-01, -1.3e-01,  1.3e-03],
       [ 0.0e+00,  0.0e+00,  0.0e+00,  0.0e+00,  0.0e+00]])